# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import json
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepaths = song_files

In [6]:
data = []
for filepath in filepaths:
    with open(filepath, 'r') as f:
        data.append(json.load(f))
df = pd.DataFrame(data) 
df.head()
# with open(filepath, 'r') as f:
#     data = json.load(f)
# df = pd.DataFrame([data]) 
# df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARGSAFR1269FB35070,NaN,,NaN,Blingtones,29.54404,1,SOTCKKY12AB018A141,Sonnerie lalaleulé hi houuu,0
1,AR7SMBG1187B9B9066,NaN,,NaN,Los Manolos,218.38322,1,SOBCOSW12A8C13D398,Rumba De Barcelona,0
2,ARBEBBY1187B9B43DB,NaN,"Gainesville, FL",NaN,Tom Petty,236.17261,1,SOFFKZS12AB017F194,A Higher Place (Album Version),1994
3,ARGCY1Y1187B9A4FA5,36.16778,"Nashville, TN.",-86.77836,Gloriana,153.33832,1,SOQOTLQ12AB01868D0,Clementina Santafè,0
4,ARAGB2O1187FB3A161,NaN,,NaN,Pucho & His Latin Soul Brothers,338.23302,1,SOLEYHO12AB0188A85,Got My Mojo Workin,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id','title', 'artist_id', 'year', 'duration']]
song_data = song_data.values[0]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
df = df[['artist_id','artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = df.values[0]
print(artist_data)

['ARGSAFR1269FB35070' 'Blingtones' '' nan nan]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath = log_files[0]
print(filepath)

/home/ammad/Udacity/P1/project_template/data/log_data/2018/11/2018-11-09-events.json


In [13]:
with open(filepath, 'r') as f:
    df = pd.read_json(f, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df_ns = df[df['page'].str.contains("NextSong")]
df_ns.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [15]:
t = pd.to_datetime(df_ns['ts'], unit='ms')
t.head()

0   2018-11-09 00:06:17.796
1   2018-11-09 00:09:46.796
2   2018-11-09 00:12:27.796
3   2018-11-09 00:14:52.796
4   2018-11-09 00:17:44.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_hour= t.dt.hour
time_second = t.dt.second
time_data = list(zip(t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year ,t.dt.weekday))
#dtt = pd.DataFrame(time_data, columns=['Hour','Day'])
# print(time_data)
column_labels = ['timestamp','hour', 'day', 'weekofyear', 'month', 'year' , 'weekday']

In [17]:
time_df = pd.DataFrame(time_data, columns=column_labels)

time_df.head()

,timestamp,hour,day,weekofyear,month,year,weekday
0,2018-11-09 00:06:17.796,0,9,45,11,2018,4
1,2018-11-09 00:09:46.796,0,9,45,11,2018,4
2,2018-11-09 00:12:27.796,0,9,45,11,2018,4
3,2018-11-09 00:14:52.796,0,9,45,11,2018,4
4,2018-11-09 00:17:44.796,0,9,45,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId','firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,42,Harper,Barrett,M,paid
1,42,Harper,Barrett,M,paid
2,42,Harper,Barrett,M,paid
3,42,Harper,Barrett,M,paid
4,42,Harper,Barrett,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    print( (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None
    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

('Supermassive Black Hole (Twilight Soundtrack Version)', 'Muse', 209.50159)
('Lighten Up', 'Beastie Boys', 161.56689)
('Pienso En Ti', 'Shakira', 145.84118)
('Amor Prohibido', 'Selena', 172.66893)
("Day 'N' Nite", 'Kid Cudi Vs Crookers', 162.97751)
('Black Masks & Gasoline', 'Rise Against', 179.59138)
("I'm Like A Lawyer With The Way I'm Always Trying To Get You Off (Me & You)", 'Fall Out Boy', 211.69587)
('Staple It Together', 'Jack Johnson', 196.12689)
('Slow Jamz (Feat. Kanye West & Jamie Foxx) (Edited Album Version)', 'Twista feat. Kayne West & Jamie Foxx', 212.55791)
('Hey_ Soul Sister', 'Train', 216.76363)
('Rehab', 'Rihanna', 293.82485)
('Van Helsing Boombox', 'Man Man', 222.45832)
('The Stallion', 'Ween', 276.13995)
('Fireflies', 'Charttraxx Karaoke', 225.17506)
('I Want I Want', 'Digitalism', 208.24771)
('Railway', 'Dispatch', 247.50975)
('Clark Gable (Album)', 'Postal Service', 294.47791)
('Phaeton', 'Patricia Barber', 317.83138)
('Ucci Ucci feat. Nicki Minaj & The Chopper C

('Natural Anthem (Album)', 'Postal Service', 307.53914)
('Love On The Rocks', 'Sara Bareilles', 253.02159)
('Pass You By', 'Gillian Welch', 237.26975)
(None, None, nan)
('Gone', 'M83', 367.5424)
("Unless It's Kicks", 'Okkervil River', 278.54322)
('DNA', '1200 Mics', 483.39546)
(None, None, nan)
('Crazy Beautiful', 'Chasen', 166.16444)
('Sonrisa', 'Jonas Steur', 481.38404)
('Call Me', 'Franz Ferdinand', 204.14649)
('Kromozom Bankal (Linathaleyco Mix)', 'Recycler', 323.52608)
(None, None, nan)
('Pea (Album Version)', 'Red Hot Chili Peppers', 107.31057)
('Red Elephant', 'Sunny Day Real Estate', 198.97424)
(None, None, nan)
('Your Decision', 'Alice In Chains', 282.30485)
('Stripes', 'Bell Orchestre', 137.82159)
('Bleed It Out [Live At Milton Keynes]', 'Linkin Park', 494.99383)
("Ain't Misbehavin", 'Sam Cooke', 122.04363)
('Every Shining Time You Arrive (Album)', 'Sunny Day Real Estate', 253.43955)
("I'm Shady", 'Eminem', 212.00934)
('Call 9-1-1 (Explicit)', 'Westside Connection', 223.60771

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.